# Energy Use Data Analysis / Visualization

# Introduction
This project is meant to gather insights on electricity usage. This file will visualize the cleaned data.

## Data
This takes in the file outputted from the "green_button_data_cleaning.ipynb".


## Original Data Source
Data is from my energy company(ComEd) from the past year. 10_22_2022 to 10_22_2023
Data from the [My Green Button](https://secure.comed.com/MyAccount/MyBillUsage/pages/secure/GreenButtonConnectDownloadMyData.aspx) webpage on the ComEd website.

# Column / header info
* **TYPE**: Electric usage
* **DATE**: Day recorded
* **START TIME**: start of recording in Hour:Minutes
* **END TIME**: end of recording in Hour:Minutes
* **USAGE**: Electric usage in kWh
* **UNITS**: Electric usage metric
* **COST**: amount charged for energy usage in USD
* **NOTES**: useless data

In [1]:
import pandas as pd
import numpy as np

In [6]:
# Import the energy use spreadsheet from the 'data' directory

# Define the directory path and the regular expression pattern
import glob
directory_path = "./data"
file_pattern = "clean_energy_use*.csv"

# Use glob.glob to match filenames based on the pattern
file_path = glob.glob(f"{directory_path}/{file_pattern}")[0]
energy_df = pd.read_csv(filepath_or_buffer=file_path)

In [7]:
print(energy_df.columns)
energy_df.head()

Index(['START_TIME', 'DATE', 'END_TIME', 'USAGE', 'COST', 'USAGE_DUR'], dtype='object')


,START_TIME,DATE,END_TIME,USAGE,COST,USAGE_DUR
0,2022-10-22 00:00:00,2022-10-22 00:00:00,2022-10-22 00:29:00,0.11,0.01,0 days 00:29:00
1,2022-10-22 00:30:00,2022-10-22 00:00:00,2022-10-22 00:59:00,0.13,0.02,0 days 00:29:00
2,2022-10-22 01:00:00,2022-10-22 00:00:00,2022-10-22 01:29:00,0.09,0.01,0 days 00:29:00
3,2022-10-22 01:30:00,2022-10-22 00:00:00,2022-10-22 01:59:00,0.20,0.02,0 days 00:29:00
4,2022-10-22 02:00:00,2022-10-22 00:00:00,2022-10-22 02:29:00,0.10,0.01,0 days 00:29:00


In [70]:
energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TYPE        17520 non-null  object 
 1   DATE        17520 non-null  object 
 2   START TIME  17520 non-null  object 
 3   END TIME    17520 non-null  object 
 4   USAGE       17520 non-null  float64
 5   UNITS       17520 non-null  object 
 6   COST        17520 non-null  object 
 7   NOTES       0 non-null      float64
dtypes: float64(2), object(6)
memory usage: 1.1+ MB


# Graph Ideas
* bucket usage by week
* bucket usage by hour
* buckec the previous graphs by season